In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from xml_script import select_car_frames, longest_car_id

import drjit as dr
import mitsuba as mi
import numpy as np
import xml.etree.ElementTree as ET

# Import or install Sionna
try:
    import sionna.rt
except ImportError as e:
    import os
    os.system("pip install sionna-rt")
    import sionna.rt

no_preview = False # Toggle to False to use the preview widget
                  # instead of rendering for scene visualization

from sionna.rt import load_scene, PlanarArray, Transmitter, Receiver, Camera,\
                      PathSolver, ITURadioMaterial, SceneObject

In [2]:
scene = load_scene('custom_scene2.xml') # Objects are merged by default
# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="iso",
                             polarization="V")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="iso",
                             polarization="cross",
                             )

# Create transmitter

car_material = ITURadioMaterial("car-material",
                                "metal",
                                thickness=0.01,
                                color=(0.8, 0.1, 0.1))


scene.preview();

In [29]:

X_max = 1713.93
Y_max = 1328.26

# car = SceneObject(fname=sionna.rt.scene.low_poly_car, name='car',radio_material=car_material)
# scene.edit(add=car)
# car.position = mi.Point3f(34.81-X_max/2, 438.98 -Y_max/2,2)
car_material = ITURadioMaterial("car-material",
                                "metal",
                                thickness=0.01,
                                color=(0.8, 0.1, 0.1))


In [30]:

filename = 'data_cars.xml'
tree  = ET.parse(filename)
root = tree.getroot()
car_id = f'{longest_car_id(filename = filename)}'
start_frame, stop_frame = select_car_frames(filename = filename, car_id=car_id)

for stamp in root:
    vehicle = stamp.find('vehicle')
    
    if (vehicle!=None) and (float(stamp.attrib.get('time')) >= start_frame) and (float(stamp.attrib.get('time')) <= stop_frame):
        print(float(stamp.attrib.get('time')))
        veh_arr=[]
        for i in range(len(stamp)):
            veh_arr.append({'vehId':int(stamp[i].attrib.get('id')[3:]),
                        'x_coor': float(stamp[i].attrib.get('x')),
                        'y_coor': float(stamp[i].attrib.get('y')),
                        'angle' :float(stamp[i].attrib.get('angle'))})
            

        cars = [SceneObject(fname=sionna.rt.scene.low_poly_car,name=f'car{veh_arr[i]['vehId']}',radio_material=car_material) for i in range(len(veh_arr))]
        for i in range(len(veh_arr)):
            if int(veh_arr[i].get('vehId')) == int(car_id):
                main_car_id = i
                break
        cam = Camera(position=[veh_arr[main_car_id]['x_coor']-X_max/2,veh_arr[main_car_id]['y_coor']-Y_max/2,50],look_at=[veh_arr[main_car_id]['x_coor']-X_max/2,veh_arr[main_car_id]['y_coor']-Y_max/2,0])
        scene.edit(add=cars)

        tx_arr=[]
        for i in range(len(veh_arr)):
            cars[i].position = mi.Point3f(veh_arr[i]['x_coor']-X_max/2, veh_arr[i]['y_coor'] - Y_max/2, 1)
            cars[i].orientation = mi.Point3f(float(-np.radians(veh_arr[i]['angle']+90)),0,0)
            cars[i].scaling = mi.Float(1.5)
            
            
            if i == main_car_id:
                scene.add(Receiver(f'rx-{i}',position=[veh_arr[i]['x_coor']-X_max/2, veh_arr[i]['y_coor'] - Y_max/2, 3],display_radius=2))
            else:
                scene.add(Transmitter(f'tx-{i}', position=[veh_arr[i]['x_coor']-X_max/2, veh_arr[i]['y_coor'] - Y_max/2, 3],display_radius=2))
        p_solver = PathSolver()
        paths = p_solver(scene=scene,
                         max_depth=3,
                         los=True,
                         specular_reflection=True,
                         diffuse_reflection=False,
                         refraction=True,
                         synthetic_array=False,
                         seed=42)
        try:
            scene.render_to_file(camera='preview',
                             filename=f'results/render_ver3/frame{stamp.attrib.get('time')}.png',
                             resolution=[650,500],
                             paths=paths);
        except Exception as e:
            print(e)
            scene.render_to_file(camera='preview',
                             filename=f'results/render_ver3/frame{stamp.attrib.get('time')}.png',
                             resolution=[650,500]);
        for i in range(len(veh_arr)):
            scene.remove(f'tx-{i}')
            if i == main_car_id:
                scene.remove(f'rx-{i}')
        scene.edit(remove=cars)  
            

39.0


/home/chartist/Desktop/sionna/sionna-venv/lib/python3.12/site-packages/drjit/ast.py:838: RuntimeWarning: The AST-transforming decorator @drjit.syntax was called more than 1000 times by your program. Since transforming and recompiling Python code is a relatively expensive operation, it should not be used within loops or subroutines. Please move the function to be transformed to the top program level and decorate it there.
  warnings.warn(


40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
